### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment. 

👉 **Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [ ]:
import utils
import time
from apimtypes import *
from apimtypes import Role

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.SIMPLE_APIM
tags        = ['authX', 'jwt', 'hr']       # ENTER DESCRIPTIVE TAG(S)
api_prefix  = 'authX-'               # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES

# 2) Service-defined parameters (please do not change these)
rg_name       = utils.get_infra_rg_name(deployment, index)
sample_folder = "authX"
nb_helper     = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM], True)

# 3) Define the APIs and their operations and policies

# Policies
# Named values must be set up a bit differently as they need to have two surrounding curly braces
hr_all_operations_xml = utils.read_policy_xml('hr_all_operations.xml', sample_name = sample_folder).format(
    jwt_signing_key = '{{' + nb_helper.jwt_key_name + '}}', 
    hr_member_role_id = '{{HRMemberRoleId}}'
)
hr_get_xml = utils.read_policy_xml('hr_get.xml', sample_name = sample_folder).format(
    hr_administrator_role_id = '{{HRAdministratorRoleId}}',
    hr_associate_role_id = '{{HRAssociateRoleId}}'
)
hr_post_xml = utils.read_policy_xml('hr_post.xml', sample_name = sample_folder).format(
    hr_administrator_role_id = '{{HRAdministratorRoleId}}'
)

# Employees (HR)
hremployees_get = GET_APIOperation('Gets the employees', hr_get_xml)
hremployees_post = POST_APIOperation('Creates a new employee', hr_post_xml)
hremployees = API('Employees', 'Employees', '/employees', 'This is a Human Resources API to obtain employee information', hr_all_operations_xml, operations = [hremployees_get, hremployees_post], tags = tags)

# APIs Array
apis: List[API] = [hremployees]

# 4) Set up the named values
nvs: List[NamedValue] = [
    NamedValue(nb_helper.jwt_key_name, nb_helper.jwt_key_value_bytes_b64, True),
    NamedValue('HRMemberRoleId', Role.HR_MEMBER),
    NamedValue('HRAssociateRoleId', Role.HR_ASSOCIATE),
    NamedValue('HRAdministratorRoleId', Role.HR_ADMINISTRATOR)
]

utils.print_ok('Notebook initialized')

### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'namedValues': {'value': [nv.to_dict() for nv in nvs]}
}

# 2) Deploy the bicep template
output = nb_helper.deploy_bicep(bicep_parameters)

if output.json_data:
    apim_name = output.get('apimServiceName', 'APIM Service Name')
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')

utils.print_ok('Deployment completed')

### ✅ 3. Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
import utils
from apimrequests import ApimRequests
from apimtesting import ApimTesting
from apimtypes import Role
from users import UserHelper
from authfactory import AuthFactory

tests = ApimTesting("AuthX Sample Tests", sample_folder, deployment)

# Preflight: Check if the infrastructure architecture deployment uses Azure Front Door. If so, assume that APIM is not directly accessible and use the Front Door URL instead.
endpoint_url = utils.test_url_preflight_check(deployment, rg_name, apim_gateway_url)

# 1) HR Administrator
# Create a JSON Web Token with a payload and sign it with the symmetric key from above.
encoded_jwt_token_hr_admin = AuthFactory.create_symmetric_jwt_token_for_user(UserHelper.get_user_by_role(Role.HR_ADMINISTRATOR), nb_helper.jwt_key_value)
print(f'\nJWT token for HR Admin:\n{encoded_jwt_token_hr_admin}')  # this value is used to call the APIs via APIM

# Set up an APIM requests object with the JWT token
reqsApimAdmin = ApimRequests(endpoint_url)
reqsApimAdmin.headers['Authorization'] = f'Bearer {encoded_jwt_token_hr_admin}'

# Call APIM
output = reqsApimAdmin.singleGet('/employees', msg = 'Calling GET Employees API via API Management Gateway URL. Expect 200.')
tests.verify(output, 'Returning a mock employee')

output = reqsApimAdmin.singlePost('/employees', msg = 'Calling POST Employees API via API Management Gateway URL. Expect 200.')
tests.verify(output, 'A mock employee has been created.')

# 2) HR Associate
# Create a JSON Web Token with a payload and sign it with the symmetric key from above.
encoded_jwt_token_hr_associate = AuthFactory.create_symmetric_jwt_token_for_user(UserHelper.get_user_by_role(Role.HR_ASSOCIATE), nb_helper.jwt_key_value)
print(f'\nJWT token for HR Associate:\n{encoded_jwt_token_hr_associate}')  # this value is used to call the APIs via APIM

# Set up an APIM requests object with the JWT token
reqsApimAssociate = ApimRequests(endpoint_url)
reqsApimAssociate.headers['Authorization'] = f'Bearer {encoded_jwt_token_hr_associate}'

# Call APIM
output = reqsApimAssociate.singleGet('/employees', msg = 'Calling GET Employees API via API Management Gateway URL. Expect 200.')
tests.verify(output, 'Returning a mock employee')

output = reqsApimAssociate.singlePost('/employees', msg = 'Calling POST Employees API via API Management Gateway URL. Expect 403.')
# The return value is not good enough, but checking for an empty string is the best we can do for now.
tests.verify(output, '')

tests.print_summary()

utils.print_ok('All done!')